In [53]:
import requests
import pandas as pd
import numpy as np
import time
import oracledb
import getpass
from geopy.geocoders import Nominatim

# Connect to database
password = getpass.getpass("DB Password: ")
wallet_password = getpass.getpass("Wallet Password: ")
connection = oracledb.connect(
    user="admin",
    password=password,
    dsn = "ltl3y0m4d7of29l1_high",
    config_dir="./config",
    wallet_location="./config",
    wallet_password=wallet_password)

print("Successfully connected to Oracle Database")
cursor = connection.cursor()

# Define functions for geolocation
geolocator = Nominatim(user_agent="MU-DS-Capstone")
def fix_address(address):
    #Error catching
    #Not-string error
    address = str(address)
    # Remove city
    if "," in address:
        address = address[:address.index(",")].strip()
    # If intersection, just use first street. TODO: Improve this
    if "/" in address:
        address = address[:address.index("/")].strip()
    # Remove NA values
    if pd.isna(address):
        address = np.nan
    # Change block address to just be that address number
    address = address.replace("-BLK", "")
    # Catching the BLVD error, needs to say BLVD not BL
    if address.endswith("BL"):
        address += "VD"
    # Catching MLK DR error, use old name
    address = address.replace("MARTIN L KING JR DR", "OLD WORLD THIRD ST")
    # Layton Error
    address = address.replace("S LAYTON ST", "S LAYTON BLVD")
    # Leon Error
    address = address.replace("LEON TR", "LEON TERRACE")
    # Mc Kinley Error
    address = address.replace("MC KINLEY", "MCKINLEY")
    # W Fond Du Lac Error
    if "FOND DU LAC" in address and "AV" not in address:
        address = address.replace("FOND DU LAC", "FOND DU LAC AV")
    # Bluemound road error
    address = address.replace("BLUE MOUND RD", "BLUEMOUND RD")        
    return address + " MILWAUKEE"

def geocode(address):
    try:
        geocoded = geolocator.geocode(address)
    except Exception as e:
        print("Error geocoding", e)
    return geocoded

def get_gps(geocoded, index):
    if geocoded is None:
        return np.nan
    gps = geocoded[1][index]
    if index == 0 and abs(gps - 43.0389) > 1:
        return np.nan
    elif index == 1 and abs(gps + 87.9065) > 1:
        return np.nan  
    return geocoded[1][index] 

# Run indefinitely
while True:
    # Grab police call log from HTML table
    url = 'https://itmdapps.milwaukee.gov/MPDCallData/index.jsp?district=All'
    headers = {'User-Agent': 'Marquette Data Science'}
    html = requests.get(url, headers=headers).content
    df = pd.read_html(html, index_col=0, header=None)[-1]
    # Remove multi-index
    df.columns = df.columns.droplevel(0)

    # Add Latitude and Longitude columns to dataframe
    fixed_addresses = df.Location.transform(fix_address)
    geocoded_addresses = fixed_addresses.apply(geocode)
    df["Latitude"] = geocoded_addresses.apply(get_gps, index=0)
    df["Longitude"] = geocoded_addresses.apply(get_gps, index=1)

    """ SEND DATA TO SQL DATABASE """
    
    for callnumber, (date, location, district, nature, status, latitude, longitude) in df.iterrows():
        if pd.isna(latitude): latitude = "NULL"
        if pd.isna(longitude): longitude = "NULL"
        try:
            query = f"SELECT * FROM CALLS WHERE CALL_NUMBER = '{callnumber}' AND DATE_TIME = '{date}' AND LOCATION = '{location}' \
            AND POLICE_DISTRICT = '{district}' AND NATURE_OF_CALL = 'nature' AND STATUS = 'Service in Progress'"
            response = cursor.execute(query).fetchall()
            if response:
                continue # SKIP IF ALREADY EXISTS IN DATABASE
            query = f"INSERT INTO CALLS VALUES ('{callnumber}', '{date}', '{location}', '{district}', '{nature}', '{status}', {latitude}, {longitude})"
            cursor.execute(query)
        except Exception as e:
            print("Error updating Database:", e)
            print(callnumber, date, location, district, nature, status, latitude, longitude)

    connection.commit()
    
    time.sleep(30*60) # Sleep for 30 minutes

DB Password:  ················
Wallet Password:  ················


Exception ignored in: <function Connection.__del__ at 0x7fb9e3e3d820>
Traceback (most recent call last):
  File "/Users/school/opt/anaconda3/lib/python3.9/site-packages/oracledb/connection.py", line 151, in __del__
    self._impl.close(in_del=True)
  File "src/oracledb/impl/thin/connection.pyx", line 307, in oracledb.thin_impl.ThinConnImpl.close
  File "src/oracledb/impl/thin/protocol.pyx", line 86, in oracledb.thin_impl.Protocol._close
  File "src/oracledb/impl/thin/protocol.pyx", line 121, in oracledb.thin_impl.Protocol._close
  File "src/oracledb/impl/thin/protocol.pyx", line 298, in oracledb.thin_impl.Protocol._final_close
  File "/Users/school/opt/anaconda3/lib/python3.9/ssl.py", line 1280, in shutdown
    super().shutdown(how)
OSError: [Errno 49] Can't assign requested address


Successfully connected to Oracle Database


DatabaseError: DPY-4011: the database or network closed the connection
[Errno 32] Broken pipe

Exception ignored in: 'oracledb.thin_impl.Protocol._send_marker'
Traceback (most recent call last):
  File "src/oracledb/impl/thin/packet.pyx", line 612, in oracledb.thin_impl.WriteBuffer.end_request
  File "src/oracledb/impl/thin/packet.pyx", line 591, in oracledb.thin_impl.WriteBuffer._send_packet
  File "/Users/school/opt/anaconda3/lib/python3.9/site-packages/oracledb/errors.py", line 111, in _raise_err
    raise exc_type(_Error(message)) from cause
oracledb.exceptions.DatabaseError: DPY-4011: the database or network closed the connection
[Errno 32] Broken pipe


DatabaseError: DPY-4011: the database or network closed the connection
[Errno 32] Broken pipe

Exception ignored in: 'oracledb.thin_impl.Protocol._send_marker'
Traceback (most recent call last):
  File "src/oracledb/impl/thin/packet.pyx", line 612, in oracledb.thin_impl.WriteBuffer.end_request
  File "src/oracledb/impl/thin/packet.pyx", line 591, in oracledb.thin_impl.WriteBuffer._send_packet
  File "/Users/school/opt/anaconda3/lib/python3.9/site-packages/oracledb/errors.py", line 111, in _raise_err
    raise exc_type(_Error(message)) from cause
oracledb.exceptions.DatabaseError: DPY-4011: the database or network closed the connection
[Errno 32] Broken pipe


Error updating Database: DPY-4011: the database or network closed the connection
230381085 02/07/2023 09:58:55 PM 6100 W LISBON AV,MKE 7 BATTERY DV Unable to Locate Complainant 43.0692162 -87.98900044999999
Error updating Database: DPY-4011: the database or network closed the connection
[Errno 9] Bad file descriptor
230381230 02/07/2023 09:57:58 PM 4054 N 64TH ST,MKE 7 THREAT Advised 43.09127895 -87.9917456248324
Error updating Database: DPY-4011: the database or network closed the connection
[Errno 9] Bad file descriptor
230381309 02/07/2023 09:57:53 PM 3008 S 8TH ST,MKE 6 CALL FOR POLICE Advised 42.9898583 -87.9212466958434
Error updating Database: DPY-4011: the database or network closed the connection
[Errno 9] Bad file descriptor
230381269 02/07/2023 09:57:34 PM 1766 W LAYTON AV,MKE 6 TRBL W/SUBJ Service in Progress 42.95922231313131 -87.93591145454545
Error updating Database: DPY-4011: the database or network closed the connection
[Errno 9] Bad file descriptor
230381317 02/07/202

DatabaseError: DPY-4011: the database or network closed the connection
[Errno 9] Bad file descriptor